# 1. Opening of link, scrolling down to the bottom, downloading the entire page and closing the browser.

In [1]:
from selenium import webdriver
#from selenium.webdriver.common.keys import Keys
#from selenium.common.exceptions import TimeoutException
#from selenium.webdriver.common.by import By
from time import sleep
#import selenium.webdriver.support.ui as ui
#import selenium.webdriver.support.expected_conditions as EC
#import os
import time
from bs4 import BeautifulSoup
import unicodedata
import xlsxwriter
import re

#set up browser, make chrome_path equal to location of your chromedriver.exe
options = webdriver.ChromeOptions()
#options.add_argument('headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--user-agent=""Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.157 Safari/537.36""')
prefs = {"profile.managed_default_content_settings.images":2,
         "profile.default_content_setting_values.notifications":2,
         "profile.managed_default_content_settings.stylesheets":2,
         "profile.managed_default_content_settings.cookies":2,
         "profile.managed_default_content_settings.javascript":1,
         "profile.managed_default_content_settings.plugins":1,
         "profile.managed_default_content_settings.popups":2,
         "profile.managed_default_content_settings.geolocation":2,
         "profile.managed_default_content_settings.media_stream":2,
        }
options.add_experimental_option("prefs",prefs)
chrome_path = r"C:\Users\salmanmirza\anaconda3\chromedriver.exe"
driver = webdriver.Chrome(executable_path = chrome_path, options=options)
#the site which has the long scrolling page
driver.get("https://ythisnews.com/aur/category/hyderabad")
#scrolling down till the botttom
lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match=False
while(match==False):
                lastCount = lenOfPage
                time.sleep(3)
                lenOfPage = driver.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
                if lastCount==lenOfPage:
                    match=True
                    
#downloading whole page
soup_a = BeautifulSoup(driver.page_source, "lxml")
print("Downloaded entire page.")
#closing browser/driver
driver.close()

Downloaded entire page.


# 2. Finding and scraping desired data i.e. extracting URLs of all headings.

In [2]:
containers = soup_a.findAll("div",{"class":"tdb_module_loop td_module_wrap td-animation-stack"})
urls = []
for container in containers:
    h3_tags = container.find_all("h3",{"class":"entry-title td-module-title"})
    url = h3_tags[0].a['href']
    urls.append(url)

total_urls = len(urls)
print("Total number of URLs: {}".format(total_urls))

Total number of URLs: 755


# 3. Extracting headline and description of news article by opening URL and writing into the excel file.

In [ ]:
#headings = []
#descs = []
#f = open("telangana.xlsx", "a", encoding="utf-16")

excelFile = xlsxwriter.Workbook("hyderabad.xlsx")
excelSheet = excelFile.add_worksheet()
index = 0
for url_num in range(0, total_urls):
    options.add_argument('headless')
    driver = webdriver.Chrome(executable_path = chrome_path, options=options)
    print("Working on {}: {}".format(url_num, urls[url_num]))
    driver.get(urls[url_num])
    soup_b = BeautifulSoup(driver.page_source, "lxml")
    driver.close()
    containers = soup_b.find_all("div",{"class":"tdc-row"})
    #heading = containers[1].find("h1",{"class":"tdb-title-text"}).text
    #f.write(heading + "\n")
    #headings.append(heading)
    #excelSheet.write(index, 0, heading)
    #index = index + 1
    paragraphs = containers[1].div.div.div.find_all("p")
    #desc = []
    for para in paragraphs:
        #p = unicodedata.normalize("NFKD", para.text)
        p = para.text.strip()
        if p == "":
            continue
        p = p.replace('\n','')
        sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', p)
        for sentence in sentences:
            excelSheet.write(index, 0, sentence)
            index = index + 1
        #f.write(p + "\n")
        #desc.append(p)
    #descs.append(desc)
else:
    print("\nFinally finished!") 

excelFile.close()
    
#f.close()
#print(headings)
#print(descs)

Working on 0: https://ythisnews.com/aur/hyderabad-selaab-se-mutasirah-ilaqon-mein-log-fungal-infection-aur-diarrhea-ka-shikaar
Working on 1: https://ythisnews.com/aur/barisho-ke-10-din-baad-bhi-hyderabad-ke-nashebi-ilaqay-hanooz-pani-mein-mahsor
Working on 2: https://ythisnews.com/aur/hyderabad-ko-corona-se-aamdani-mutaser-hone-walay-shehron-mein-teesra-maqam
Working on 3: https://ythisnews.com/aur/secunderabad-ko-nazar-andaaz-karne-ki-shikayat
Working on 4: https://ythisnews.com/aur/mutasirah-hyderabadi-awam-gharon-ki-taamer-no-ke-liye-mazeed-imdaad-ke-khwahan
Working on 5: https://ythisnews.com/aur/covid-ne-li-shohar-ki-jaan-biwi-ne-ki-khudkushi
Working on 6: https://ythisnews.com/aur/jo-engineers-nahi-karpaye-woh-karnaama-qadeer-ne-kar-dekhaya
Working on 7: https://ythisnews.com/aur/nizam-ahad-ka-kamaan-munhadim-kardiya-gaya
Working on 8: https://ythisnews.com/aur/durgam-chevuru-cable-bridge-selfie-lainay-par-2000-jurmana
Working on 9: https://ythisnews.com/aur/selaab-se-mutasirah-7

Working on 80: https://ythisnews.com/aur/urdu-university-ke-regular-course-mein-dakhlay-jari
Working on 81: https://ythisnews.com/aur/faiz-ne-meer-ghalib-aur-iqbal-ki-sheri-rewayton-ki-towsee-ki
Working on 82: https://ythisnews.com/aur/shadion-mein-bin-bulaye-maheman-corona-ko-nah-bhoolain
Working on 83: https://ythisnews.com/aur/hyderabad-ke-jubilee-hills-flyover-iftitah-ke-liye-tayyar
Working on 84: https://ythisnews.com/aur/covid19-ne-fashion-aur-parcha-jaat-ki-sanat-ko-mutasir-kiya
Working on 85: https://ythisnews.com/aur/hyderabad-mein-bunkron-ke-liye-kaam-nahi-watan-wapes
Working on 86: https://ythisnews.com/aur/teraaki-ka-shoq-shiva-ney-100-afraad-ko-khudkushi-se-bachaaya
Working on 87: https://ythisnews.com/aur/hyderabad-aur-telangana-ke-baz-degar-ilaaqon-mein-barish
Working on 88: https://ythisnews.com/aur/hyd-ke-sadkon-ki-thila-bandi-ke-karobar-par-corona-ka-asar
Working on 89: https://ythisnews.com/aur/zayed-bill-par-private-hospital-par-high-court-ki-narazgi
Working on 90: 

Working on 157: https://ythisnews.com/aur/hyderabad-ki-3-companies-nasa-ke-ventilator-ki-tayari-ke-liye-muntakhib
Working on 158: https://ythisnews.com/aur/khangi-hostels-ka-dewalia-muashi-bohraan-se-nimatne-ki-nakaam-koshish
Working on 159: https://ythisnews.com/aur/lockdown-ke-3-haftay-baad-bhi-it-companian-hanooz-weraan
Working on 160: https://ythisnews.com/aur/inter-ke-nataij-ka-15-ya-17-june-ko-elaan-mutawaqqa
Working on 161: https://ythisnews.com/aur/is-saal-1-lakh-se-ziyada-talba-ko-10-10-gpa-mutawaqqa
Working on 162: https://ythisnews.com/aur/hotels-ko-mulazimeen-ki-qillat-ka-saamna-menu-se-kayi-khanay-gayab
Working on 163: https://ythisnews.com/aur/khangi-financer-ne-auto-drivers-ki-pareshaaniya-badhaden
Working on 164: https://ythisnews.com/aur/mohammed-muntazim-ko-education-mein-p-hd
Working on 165: https://ythisnews.com/aur/santosh-nagar-flyover-bridge-awam-ke-liye-masail-ka-markaz
Working on 166: https://ythisnews.com/aur/khasta-haal-imaartoun-ko-munhadim-karne-ghmc-ne-man

Working on 230: https://ythisnews.com/aur/sania-mirza-ne-wazan-kam-karne-ka-raaz-faash-kardiya
Working on 231: https://ythisnews.com/aur/maktuta-shanasi-par-ek-roza-work-shop-ka-ineqad
Working on 232: https://ythisnews.com/aur/golconda-masters-golf-championship-ka-hyderabad-mein-aaghaz
Working on 233: https://ythisnews.com/aur/ghmc-ab-marghi%e2%80%ac-chowk-tazaeen-ki-taraf-tawajah-markooz-kere-gi
Working on 234: https://ythisnews.com/aur/telangana-mein-coronavirus-se-koi-mutasir-nahi-sardi-zukam-aam-noiyat-ke-amraaz
Working on 235: https://ythisnews.com/aur/hyderabad-mein-macharoun-ke-khilaaf-drone-hamlay-ekhtraay-koshish-ke-hosla-afzahi
Working on 236: https://ythisnews.com/aur/telangana-mein-tanashahi-urooj-par-awam-se-ehtijaj-ka-haq-cheennay-ki-koshish
Working on 237: https://ythisnews.com/aur/police-ne-caa-ki-khilaaf-khawateen-ke-ehtajaji-muzahirey-ki-ijazat-se-kiya-inkaar
Working on 238: https://ythisnews.com/aur/khilwat-ground-par-caa-ke-khilaaf-ehtejaji-jalsa-wo-mushaira-naamwar

Working on 300: https://ythisnews.com/aur/kcr-ne-krtc-mulazmeen-par-tohfoun-ki-bo-chhar-ijlaas-mein-kiya-mukhtalif-sahulatoun-ka-elaan
Working on 301: https://ythisnews.com/aur/khud-ho-ke-khatakar-nikla-tha-maqtool-dr-se-insaaf-ka-mutalba-karne
Working on 302: https://ythisnews.com/aur/rythu-bazaar-mein-40-rupay-fi-kelo-farokht-ki-jaye-gi-pyaaz
Working on 303: https://ythisnews.com/aur/mir-alam-tank-park-mein-musical-water-fountains-taamer-karne-ka-elaan
Working on 304: https://ythisnews.com/aur/hyderabad-mein-30-november-ko-salana-kul-hind-mehfil-naat-shareef%ef%b7%ba
Working on 305: https://ythisnews.com/aur/manuu-mein-yom-e-dastoor-ka-ineqad
Working on 306: https://ythisnews.com/aur/rtc-bus-ki-takkar-se-tcs-mein-kaam-karne-wali-khatoon-ki-mout
Working on 307: https://ythisnews.com/aur/helmet-hi-nahi-balkay-isi-nishaan-wali-helmet-pehnu-awam-pareshan
Working on 308: https://ythisnews.com/aur/shehar-ki-flyovers-race-track-may-tabdeel-gariyaan-bhagane-ka-shoq-insani-jaanu-ka-dushman
Wo

Working on 373: https://ythisnews.com/aur/shadeed-barish-ka-silsila-26-september-tak-barqarar-rehne-ki-pesh-qiyaasi
Working on 374: https://ythisnews.com/aur/tareekhi-fateh-darwaaza-ki-safai-behtar-iqdaam
Working on 375: https://ythisnews.com/aur/hukoomat-metro-station-haadsa-mein-halaak-khatoon-ke-ahal-e-khanah-ko-20-laakh-muawza-degi
Working on 376: https://ythisnews.com/aur/metro-station-ki-deewar-se-plaster-ke-tukde-girney-se-khatoon-ki-mout
Working on 377: https://ythisnews.com/aur/telangana-assembly-ijlaas-ka-ekhtetam-akbaruddin-owaisi-public-accounts-committee-ke-chairman-naamzad
Working on 378: https://ythisnews.com/aur/jahez-ke-liye-harasan-karne-ke-muqadmay-mein-cctv-footage-se-naya-modh-aaya
Working on 379: https://ythisnews.com/aur/mohammad-azhar-uddin-ne-hca-ke-sadaarti-ohday-ke-liye-kaghzaat-namzadgi-daakhil-kiya
Working on 380: https://ythisnews.com/aur/puranay-shehar-mein-jald-hi-metro-rail-shuru-hogi-ktr
Working on 381: https://ythisnews.com/aur/hyderabad-ke-hospitals-

Working on 444: https://ythisnews.com/aur/eid-ul-adha-ke-mauqe-par-hukumat-shehar-mein-safai-ke-intezamat-kare-asaduddin-owaisi
Working on 445: https://ythisnews.com/aur/eid-ul-adha-ki-namaz-silsila-mein-traffic-police-ki-janib-se-lagaye-gaye-traffic-zawabit
Working on 446: https://ythisnews.com/aur/mehar-organisation-ki-janib-se-yousuf-jaffar-mehar-ali-ki-119th-youm-e-paidaish-hyd-may-manaya-jaega
Working on 447: https://ythisnews.com/aur/hukoomat-tamam-ahal-sahafiyoun-ko-accreditation-card-degi-allam-narayana
Working on 448: https://ythisnews.com/aur/hyderabad-se-doha-indigo-ki-ek-aur-raast-parwaaz-ka-aaghaz
Working on 449: https://ythisnews.com/aur/eid-ke-pur-aman-ineqad-ko-yaqeeni-bananay-ke-liye-city-police-ney-kiye-ahem-iqdamaat
Working on 450: https://ythisnews.com/aur/markaz-ki-janib-se-jari-kardah-alert-ke-baad-rgi-airport-par-hifazati-intizama-mein-izafah
Working on 451: https://ythisnews.com/aur/urdu-aur-hyderabad-university-mein-zayed-police-tainaat
Working on 452: https://

Working on 514: https://ythisnews.com/aur/osmania-university-mein-digital-india-programme-ke-tehat-wifi
Working on 515: https://ythisnews.com/aur/baarish-ka-paani-aur-hitech-city-mein-traffic-jaam-companiyon-ke-auqaat-mein-tabdeeli-ki-tajweez-mustarad
Working on 516: https://ythisnews.com/aur/kiya-aap-ke-namak-mein-zeher-hai
Working on 517: https://ythisnews.com/aur/chicken-biryani-hyderabadiyon-ki-online-bhi-pehli-pasand
Working on 518: https://ythisnews.com/aur/kolkata-mein-doctoron-par-hamla-ke-khilaaf-telangana-mein-ehtejaaj
Working on 519: https://ythisnews.com/aur/barish-nah-honay-se-school-tulba-mein-qhayee-ki-shekiyatoon-mein-izafah
Working on 520: https://ythisnews.com/aur/teez-raftaar-gaadi-chalanay-walay-khabardaar-shehar-ke-mukhtalif-maqamat-par-speed-guns-nasb
Working on 521: https://ythisnews.com/aur/hyderabad-ki-khasta-haal-imaartoun-ko-munahdim-karne-ki-karwaiyoun-ka-aaghaz
Working on 522: https://ythisnews.com/aur/osmania-university-ka-6-saal-baad-convocation
Working o

Working on 585: https://ythisnews.com/aur/hajj-sirf-musalmanoun-ke-liye-khaas-qabuliat-ke-liye-pak-maal-ikhlaas-aur-akhlak-e-sharat
Working on 586: https://ythisnews.com/aur/telangana-riyasat-se-waiting-list-ke-tehat-308-hajj-aazmin-ka-intikhab
Working on 587: https://ythisnews.com/aur/moazzam-jahi-market-ko-europee-andaaz-mein-awaami-maqam-mein-tabdeel-karne-ki-tayyariyan-urooj-par
Working on 588: https://ythisnews.com/aur/secunderabad-ke-muslim-raye-dahindgaan-ka-ittehaad-kisi-bhi-umeedwar-ki-qismat-badalne-ke-liye-kaafi
Working on 589: https://ythisnews.com/aur/bjp-ney-halqe-lok-sabha-secunderabad-se-bandaru-dattatreya-ko-kiya-nazar-andaaz-kishan-reddy-ko-banaya-umeedwar
Working on 590: https://ythisnews.com/aur/hyderabad-central-university-mein-ek-aur-hiran-ki-mout
Working on 591: https://ythisnews.com/aur/metro-rail-stations-par-e-autos-ki-khidmaat-faraham-karne-ka-elaan
Working on 592: https://ythisnews.com/aur/asaduddin-owaisi-chouthi-martaba-lok-sabha-hyderabad-ke-umidwaar
Work